## Purposee

Create Big Query Vector Store !

https://python.langchain.com/v0.2/docs/integrations/platforms/google/#bigquery-vector-search

https://cloud.google.com/bigquery/docs/vector-search-intro

Google Cloud BigQuery Vector Search lets you use GoogleSQL to do semantic search, using vector indexes for fast approximate results, or using brute force for exact results.

### Reading from GCP Bucket

In [ ]:
%pip install --upgrade --quiet  langchain-google-community[gcs]
%pip install unstructured

In [ ]:
from langchain_google_community import GCSDirectoryLoader
loader = GCSDirectoryLoader(project_name="", bucket="")  #Put GCP  your Project Name & Bucket Name
lstDoc = loader.load()
lstDoc

In [ ]:
print(type(lstDoc[0]))

In [ ]:
lstSrtContent=[doc.page_content for doc in lstDoc]
lstSrtContent

In [ ]:
%pip install --upgrade --quiet  langchain langchain-google-vertexai google-cloud-bigquery

GCP Information

In [ ]:

PROJECT_ID = ""  # Put you GCP Project ID here

# Set the project id
#! gcloud config set project {PROJECT_ID}

REGION = ""  #Put Location
DATASET = "vector_dataset"  
TABLE = "tbllight"  

In [ ]:
from langchain_google_vertexai import VertexAIEmbeddings

embedding = VertexAIEmbeddings(
    model_name="textembedding-gecko@latest", project=PROJECT_ID
)

In [ ]:
from google.cloud import bigquery

client = bigquery.Client(project=PROJECT_ID, location=REGION)
client.create_dataset(dataset=DATASET, exists_ok=True)



https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.utils.DistanceStrategy.html

https://api.python.langchain.com/en/latest/vectorstores/langchain_community.vectorstores.bigquery_vector_search.BigQueryVectorSearch.html

In [ ]:
from langchain_community.vectorstores import BigQueryVectorSearch
from langchain_community.vectorstores.utils import DistanceStrategy

store = BigQueryVectorSearch(
    project_id=PROJECT_ID,
    dataset_name=DATASET,
    table_name=TABLE,
    location=REGION,
    embedding=embedding,
    distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE,
)

In [ ]:

metadatas = [{"len": len(t)} for t in lstSrtContent]
metadatas


In [ ]:
store.add_texts(lstSrtContent, metadatas=metadatas)

In [ ]:
query='S N Bose'

In [ ]:
import json
ret_Val1 = store.similarity_search(query)
print(ret_Val1)

json_output=[{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in ret_Val1]

with open('DocSearch.json', "w") as json_file:
        json.dump(json_output, json_file, indent=4)


In [ ]:
query_vector = embedding.embed_query(query)
ret_Val2 = store.similarity_search_by_vector(query_vector)
print(ret_Val2)

json_output=[{'page_content': doc.page_content, 'metadata': doc.metadata} for doc in ret_Val2]

with open('VectorSearch.json', "w") as json_file:
        json.dump(json_output, json_file, indent=4)
